<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Google Maps - Routes API Calculate the toll costs for a given route
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Google%20Sheets/Google_Sheets_Add_items_to_Notion_databases_from_new_rows_in_Google_Sheets.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedbacks</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #googlemaps #productivity #operations #automation #jupyternotebooks

**Author:** [Antonio Georgiev](www.linkedin.com/in/antonio-georgiev-b672a325b)

This notebook does the following tasks: 
- Obtains the necessary credentials for API access.
- Configures the API endpoint URL.
- API Request: Sends a GET request to the Routes API endpoint using the provided address and parameters.
- Response Handling: Handles the API response and checks for errors or successful retrieval of the toll information.
- Data Extraction: Extracts the tolls costs from the API response and outputs them

Pre-requisite: 
1. Follow the steps in Routes_API_Setup_and_Authentication and Routes_API_Geocoding
2. Make sure to enable Routes API as it is in the "Additional APIs" section in the google cloud console.
3. Check the available countries https://developers.google.com/maps/documentation/routes/calculate_toll_fees#toll-locations

## Input

### Import libraries

In [1]:
import os
import requests
import naas

### Setup variables

In [5]:
api_secret_name = "API" # add your API key directly in the API_KEY variable or using naas secret variable
API_KEY = naas.secret.get(api_secret_name)  # Read API key from the secret variable
API_ENDPOINT = 'https://routes.googleapis.com/directions/v2:computeRoutes'
origin_lat = 42.340173523716736
origin_lng = -71.05997968330408
destination_lat = 42.075698891472804
destination_lng = -72.59806562080408

## Model

#### Request for toll information (more info on https://developers.google.com/maps/documentation/routes/calculate_toll_fees)

In [6]:
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.travelAdvisory.tollInfo,routes.legs.travelAdvisory.tollInfo'
}

payload = {
    "origin": {
        "location": {
            "latLng": {
                "latitude": origin_lat,
                "longitude": origin_lng
            }
        }
    },
    "destination": {
        "location": {
            "latLng": {
                "latitude": destination_lat,
                "longitude": destination_lng
            }
        }
    },
    "travelMode": "DRIVE",
    "extraComputations": ["TOLLS"],
    "routeModifiers": {
        "vehicleInfo": {
            "emissionType": "GASOLINE"
        },
        "tollPasses": [
            "US_MA_EZPASSMA",
            "US_WA_GOOD_TO_GO"
        ]
    }
}

response = requests.post(API_ENDPOINT, json=payload, headers=headers)

## Output

### Process the response

In [10]:
if response.status_code == 200:
    data = response.json()
    routes = data['routes']
    if routes:
        toll_info = routes[0]['travelAdvisory']['tollInfo']
        estimated_price = toll_info['estimatedPrice']
        if estimated_price:
            currency_code = estimated_price[0]['currencyCode']
            toll_cost = float(estimated_price[0]['units'])
            formatted_toll_cost = '{:.2f}'.format(toll_cost)
            print('Toll Cost:', currency_code +' '+ formatted_toll_cost)
    else:
        print('No routes found in the response.')
else:
    print('Google Maps API request failed with status code:', response.status_code)

Toll Cost: USD 4.00
